In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from __future__ import print_function

In [2]:
class L40(object):
    '''Lorenz 40 model of zonal atmospheric flow'''
    
    def __init__(self, members=1, n=40, dt=0.05, F=8):
        self.n = n
        self.dt = dt
        self.dtx = dt
        self.x = np.random.normal(0., 0.1, size=(members, n))
        self.members = members
        self.F = F
        self._history = []
        self._variance = []
    
    def dxdt(self):
        dxdt = np.zeros((self.members, self.n),'f8')
        for n in range(2,self.n-1):
            dxdt[:,n] = -self.x[:,n-2]*self.x[:,n-1] +  \
                        self.x[:,n-1]*self.x[:,n+1] - self.x[:,n] + self.F
        dxdt[:,0] = -self.x[:,self.n-2]*self.x[:,self.n-1] +  \
                self.x[:,self.n-1]*self.x[:,1] - self.x[:,0] + self.F
        dxdt[:,1] = -self.x[:,self.n-1]*self.x[:,0] + \
                self.x[:,0]*self.x[:,2] - self.x[:,1] + self.F
        dxdt[:,self.n-1] = -self.x[:,self.n-3]*self.x[:,self.n-2] + \
                            self.x[:,self.n-2]*self.x[:,0] - \
                            self.x[:,self.n-1] + self.F
        return dxdt
    
    def rk4step(self):
        h = self.dt; hh = 0.5*h; h6 = h/6.
        x = self.x
        dxdt1 = self.dxdt()
        self.x = x + hh*dxdt1
        dxdt2 = self.dxdt()
        self.x = x + hh*dxdt2
        dxdt = self.dxdt()
        self.x = x + h*dxdt
        dxdt2 = 2.0*(dxdt2 + dxdt)
        dxdt = self.dxdt()
        self.x = x + h6*(dxdt1 + dxdt + dxdt2)
    
    def store(self):
        self._history.append(self.x.mean(axis=0))
        self._variance.append(self.x.var(axis=0))
    
    @property
    def history(self):
        return np.array(self._history)
    
    @property
    def variance(self):
        return np.array(self._variance)
    
    def assimilate(self, obs, var_obs, H=None, yprime=None):
        if yprime is None:
            yprime = np.sqrt(var_obs)
        
        if H is None:
            H = np.eye(self.n)
        
        xbar = self.x.mean(axis=0)
        xprime = self.x - xbar
        
        Pb = np.sum(xprime[:, np.newaxis, :]*xprime[:, :, np.newaxis], axis=0)/(self.members-1)
        R = var_obs * np.eye(H.shape[0])
        
        PbHT = np.dot(Pb, H.T)
        HPbHT = np.dot(H, PbHT)
        K = np.dot(PbHT, np.linalg.inv(HPbHT+R))
        
        xbar_a = xbar + np.dot(K, obs-np.dot(H, xbar))
        
        xprime_a = np.zeros_like(xprime)
        for n in range(members):
            obs_prime = yprime*np.random.randn(H.shape[0])
            xprime_a[n] = xprime[n] + np.dot(K, obs_prime-np.dot(H, xprime[n]))

        self.x = xbar_a + xprime_a


def fourierH(N=40):
    H = np.zeros((N, N))
    x = np.arange(float(N))
    for mode in range(N):
        H[mode, :] = np.cos(mode * x * np.pi / float(N-1))
    return H

In [7]:
def run_group(Nassim=3):
    
    members = 50
    std_obs = 1.0
    var_obs = std_obs**2

    H = np.eye(40)
    # H = np.eye(truth.n)[::2, :]
    # H = np.eye(20, 40)
    # H = fourierH()

    Nsteps = 1000
    truth = L40(members=1)
    for n in range(500):  # Initialize truth with a mature state
        truth.rk4step()

    err = []
    for ittr in range(10):
        ensemble = L40(members=members)
        for n in range(members):
            ensemble.x[n] = truth.x + std_obs*np.random.randn(40)

        for n in range(Nsteps):
            # Step
            truth.rk4step()
            ensemble.rk4step()
            # Maybe assimilate
            if np.mod(n, Nassim)==0:
                obs = np.dot(H, truth.x[0]) + std_obs*np.random.randn(H.shape[0])
                ensemble.assimilate(obs, var_obs, yprime=None, H=H)
            # Store
            truth.store()
            ensemble.store()

        err.append(np.mean((truth.history-ensemble.history)**2, axis=-1))
    
    return err

err = run_group()

ValueError: operands could not be broadcast together with shapes (2000,40) (1000,40) 